In [3]:
from finished import kinematika
import numpy as np

In [7]:
a_min = -0.05
a_max = 0.05
a_min_deg = np.rad2deg(a_min)
a_max_deg = np.rad2deg(a_max)
a_min_deg, a_max_deg


(np.float64(-2.8647889756541165), np.float64(2.8647889756541165))

In [11]:
#Dejanski parametri skrajšano
b = 0.071589 # m
p = 0.116 # m
l_1 = 0.08254 # m
l_2 = 0.1775 # m

kinematika.deg2steps(kinematika.izracun_kotov(b, p, l_1, l_2, 0.19, 0, 0))

array([4.04822136, 4.04822136, 4.04822136])

In [13]:
kinematika.deg2steps(kinematika.izracun_kotov(b, p, l_1, l_2, 0.19, -2.8647889, 0))

array([5.46124014, 3.34798628, 3.34798628])

In [12]:
kinematika.deg2steps(kinematika.izracun_kotov(b, p, l_1, l_2, 0.19, -2.8647889, -2.8647889))

array([5.45768109, 4.55409225, 2.15517832])

In [14]:
kinematika.deg2steps(kinematika.izracun_kotov(b, p, l_1, l_2, 0.19, 2.8647889, 2.8647889))


array([2.65456498, 3.5317189 , 5.9835325 ])

In [15]:
kinematika.deg2steps(kinematika.izracun_kotov(b, p, l_1, l_2, 0.19, 2.8647889, -2.8647889))


array([2.65456498, 5.9835325 , 3.5317189 ])